## Gerenciador de Portfolio

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
from lib import lib as l

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA', '#268040', '#800B00', '#A13808', '#464CC2'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2'])

## Leitura dos dados

In [13]:
dquote = pd.read_csv('br_quotes.csv')
dstocks = pd.read_csv('br_stocks.csv')
dport = pd.read_csv('br_portfolio.csv')
dinvest = pd.read_csv('br_invest.csv')
dport['Date'] = dport['Date'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
dinvest['Data'] = dinvest['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())

In [14]:
dport

,Ticker,Buy,Shares,Buy Tax,Date,Div
0,IVVB11,130.800,10,0.400,2019-10-30,0
1,BRAX11,87.500,10,0.270,2019-11-14,0
2,IVVB11,139.030,10,0.630,2019-12-12,0


In [4]:
dflow = dinvest.groupby('ID').apply(l.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow = dflow.sort_values(by=['Rendimento anual'], ascending=False).reset_index(drop=True)
dflow

,ID,Valor inicial,Ganho,Meses,Rendimento,Rendimento mensal,Rendimento anual,Ativo,Tipo
0,101.000,875.270,81.730,1.311,9.338,7.044,126.331,True,BRAX
1,102.000,1390.930,24.570,0.393,1.766,4.551,70.585,True,IVVB
2,100.000,1308.400,107.100,1.803,8.186,4.460,68.803,True,IVVB
3,5.100,1059.610,2.300,0.492,0.217,0.442,5.433,True,Nu
4,1.000,12491.070,48.940,1.115,0.392,0.351,4.299,False,Nu
5,3.100,1989.970,9.020,1.311,0.453,0.345,4.225,True,Nu
6,2.000,715.200,2.270,0.951,0.317,0.334,4.080,False,Nu
7,4.100,10.100,0.020,0.623,0.198,0.318,3.884,False,Nu
8,1.100,10940.620,57.890,1.770,0.529,0.299,3.642,True,Nu
9,3.000,2002.290,1.620,0.459,0.081,0.176,2.137,False,Nu


## Cálculo de retorno

In [5]:
dreturn = pd.DataFrame()
current_year = dt.date.today().year
dreturn['Ticker'] = dquote['Ticker']
dreturn['Predicted'] = [0 for j in dquote['Ticker']]
weights = [0.05, 0.1, 0.15, 0.3, 0.4]
for i in range(5):
    year_i = current_year-4+i
    dreturn[str(year_i)] = (dquote[str(year_i)] - dquote[str(year_i-1)]) / dquote[str(year_i-1)]
    dreturn['Predicted'] += dreturn[str(year_i)] * weights[i]

## Screener

In [6]:
dfull = dreturn.join(dstocks.set_index('Ticker'), on='Ticker', how='inner')
dfull['Price'] = dquote['2019']

assets = ['USA', 'EU', 'BR', 'EM', 'Bonds', 'Cash', 'Commod']
industry = ['Basic Mat', 'Cons Cycl', 'Finance', 'Real Estate', 'Telecom', 'Energy', 'Indust', 'Tech', 'Cons Non-Cycl', 'Health', 'Util']
dfull['Geo'] = dfull[assets].idxmax(axis=1)
dfull['Domain'] = dfull[industry].idxmax(axis=1)
dfull['Asset'] = dfull[assets].idxmax(axis=1).apply(lambda x : x if (x == 'Bonds' or x == 'Commod') else 'Equity')

screen = ['Ticker', 'Score', 'Price', 'Return', 'Risk', 'Tax', 'YTD', 'Cat', 'Geo', 'Domain', 'Asset']
dfull['Risk'] = dfull['Volatility'].apply(lambda x : l.int2pct(x/100))
dfull['Tax'] = dfull['TER'].apply(lambda x : l.int2pct(x/100))
dfull['Return'] = dfull['Predicted'].apply(l.int2pct)
dfull['YTD'] = dfull[str(current_year)].apply(l.int2pct)
weights = [0.05, 0.5, 0.35, 0.1]
dfull['Score'] = (10+(30-dfull['Price'])/27)*weights[0] + dfull['Predicted']/0.3*10*weights[1] + (10+(7-dfull['Volatility'])/2.3)*weights[2] + (10-dfull['TER']/0.2)*weights[3]
l.change_df_prop(dfull[screen].sort_values('Score', ascending=False).reset_index(drop=True))

Ticker,Score,Price,Return,Risk,Tax,YTD,Cat,Geo,Domain,Asset
BBSD11,7.99369,95.47,29.99%,17.73%,0.5%,34.2%,3,USA,Indust,Equity
DIVO11,7.93795,69.3,30.75%,19.25%,0.5%,40.14%,3,USA,Util,Equity
IVVB11,7.84647,140.21,23.75%,12.18%,0.24%,43.09%,2,USA,Tech,Equity
BRAX11,7.3552,96.05,24.94%,17.38%,0.2%,29.45%,1,BR,Finance,Equity
BOVB11,7.17266,116.26,24.39%,17.73%,0.2%,27.13%,1,BR,Finance,Equity
BOVV11,7.12266,116.26,24.39%,17.73%,0.3%,27.13%,1,BR,Finance,Equity
PIBB11,6.92094,196.39,23.3%,17.68%,0.06%,25.97%,1.1,BR,Finance,Equity
GOVE11,6.76352,50.66,26.51%,22.55%,0.5%,31.07%,1,BR,Finance,Equity
MATB11,5.70007,36.07,19.97%,22.55%,0.5%,12.16%,4,BR,Basic Mat,Equity


## Portfolio Overview

In [7]:
dport['Price'] = dport['Ticker'].apply(lambda x : dfull[dfull['Ticker'] == x]['Price'].item())
dport['Profit'] = (dport['Price'] - dport['Buy']) * dport['Shares'] + dport['Div']
dport['Yield'] = (dport['Profit'] / (dport['Buy'] * dport['Shares']))
dport['Annual yield'] = (((1 + dport['Yield']) ** (1/(dt.date.today() - dport['Date']).apply(lambda x : x.days))) ** 365 - 1)
dport['Yearly profit'] = dport['Price'] * dport['Shares'] * dport['Annual yield']
dport['Value'] = (dport['Price'] * dport['Shares'])
dport['Keep'] = dport['Ticker'].apply(lambda x : dfull[dfull['Ticker'] == x]['TER'].item()) * dport['Price'] * dport['Shares'] / 100
dport['Sell Tax'] = (dport['Profit'] - (dport['Buy Tax'] + dport['Keep'])) * 0.15
dport['Total Tax'] = dport['Buy Tax'] + dport['Keep'] + dport['Sell Tax']
dport['Tax'] = (dport['Total Tax'] / dport['Profit']).apply(l.int2pct)
dport['Yield (Liq.)'] = ((dport['Profit'] - dport['Total Tax']) / (dport['Buy'] * dport['Shares']))
dport['Annual yield (Liq.)'] = (((1 + dport['Yield (Liq.)']) ** (1/(dt.date.today() - dport['Date']).apply(lambda x : x.days))) ** 365 - 1)
dport['Yearly profit (Liq.)'] = dport['Price'] * dport['Shares'] * dport['Annual yield (Liq.)']
dport['Yield %'] = dport['Yield'].apply(l.int2pct)
dport['Annual yield %'] = dport['Annual yield'].apply(l.int2pct)
dport['Yield (Liq.) %'] = dport['Yield (Liq.)'].apply(l.int2pct)
dport['Annual yield (Liq.) %'] = dport['Annual yield (Liq.)'].apply(l.int2pct)

In [8]:
status = ['Ticker', 'Buy', 'Shares', 'Price', 'Profit', 'Yield %']
l.change_df_prop(dport[status], 24)

Ticker,Buy,Shares,Price,Profit,Yield %
IVVB11,130.8,10,140.21,94.1,7.19%
BRAX11,87.5,10,96.05,85.5,9.77%
IVVB11,139.03,10,140.21,11.8,0.85%


In [9]:
dgroup = dport.groupby('Ticker').sum()
l.plot_pie(dgroup['Value'])

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['BRAX11', 'IVVB11'], dtype=object),
 …

In [10]:
l.plot_pie(dgroup['Profit'])

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['BRAX11', 'IVVB11'], dtype=object),
 …

In [11]:
revenue = ['Ticker', 'Annual yield %', 'Yearly profit', 'Tax', 'Annual yield (Liq.) %', 'Yearly profit (Liq.)']
l.change_df_prop(dport[revenue])

Ticker,Annual yield %,Yearly profit,Tax,Annual yield (Liq.) %,Yearly profit (Liq.)
IVVB11,52.6%,737.442,18.4%,41.49%,581.667
BRAX11,113.02%,1085.52,17.18%,87.99%,845.131
IVVB11,19.9%,278.971,43.78%,10.76%,150.888


In [12]:
dtotal = pd.DataFrame({
    'Value': [(dport['Price'] * dport['Shares']).sum()],
    'Profit': [dport['Profit'].sum()],
    'Yield': [l.int2pct(dport['Profit'].sum() / (dport['Buy'] * dport['Shares']).sum())]
})
dtotal['Yearly profit'] = dport['Yearly profit'].sum()
dtotal['Annual yield'] = (dtotal['Yearly profit'] / dtotal['Value']).apply(l.int2pct)
l.change_df_prop(dtotal, 28)

Value,Profit,Yield,Yearly profit,Annual yield
3764.7,191.4,5.36%,2101.94,55.83%


## Portfolio Simulation